In [8]:
import torch
from torch import nn
from torch.nn.init import xavier_uniform_
from torch.nn.init import constant_
import math
import torch.nn.functional as F
from enum import IntEnum
import numpy as np
# from .utils import transformer_FFN, ut_mask, pos_encode, get_clones
from torch.nn import Module, Embedding, LSTM, Linear, Dropout, LayerNorm, TransformerEncoder, TransformerEncoderLayer, \
        MultiLabelMarginLoss, MultiLabelSoftMarginLoss, CrossEntropyLoss, BCELoss, MultiheadAttention
from torch.nn.functional import one_hot, cross_entropy, multilabel_margin_loss, binary_cross_entropy
# from .que_base_model import QueBaseModel,QueEmb
from torch.utils.checkpoint import checkpoint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Dim(IntEnum):
    batch = 0
    seq = 1
    feature = 2

class UNIKT(nn.Module):
    def __init__(self, n_question=100000, n_pid=188, 
            d_model=256, n_blocks=5, dropout=0.1, d_ff=256, 
            loss1=0.5, loss2=0.5, loss3=0.5, start=50, num_layers=2, nheads=4, seq_len=1024, 
            kq_same=1, final_fc_dim=512, final_fc_dim2=256, num_attn_heads=8, separate_qa=False, l2=1e-5, emb_type="qid", emb_path="", pretrain_dim=768, cf_weight=0.3, t_weight=0.3, local_rank=1, num_sgap=None, c0=0, max_epoch=0, q_window_size=20, c_window_size=4):
        super().__init__()
        """
        Input:
            d_model: dimension of attention block
            final_fc_dim: dimension of final fully connected net before prediction
            num_attn_heads: number of heads in multi-headed attention
            d_ff : dimension for fully conntected net inside the basic block
            kq_same: if key query same, kq_same=1, else = 0
        """
        self.model_name = "unikt"
        print(f"model_name: {self.model_name}, emb_type: {emb_type}")
        self.n_question = n_question
        self.dropout = dropout
        self.kq_same = kq_same
        self.n_pid = n_pid
        self.l2 = l2
        self.model_type = self.model_name
        self.separate_qa = separate_qa
        self.emb_type = emb_type
        self.ce_loss = BCELoss()
        self.cf_weight = cf_weight
        self.t_weight = t_weight
        self.num_sgap = num_sgap
        self.q_window_size = q_window_size
        self.c_window_size = c_window_size
        # print(f"q_window_size:{self.q_window_size}")
        # print(f"c_window_size:{self.c_window_size}")

        
        self.embed_l = d_model

        self.dataset_emb = nn.Embedding(20,self.embed_l).to(device)# dataset_id embedding

        self.qa_embed = nn.Embedding(2, self.embed_l)
        
        if self.emb_type.find("pt") != -1:
            self.time_emb = nn.Embedding(self.num_sgap+1, self.embed_l)
        # Architecture Object. It contains stack of attention block

        # self.emb_q = nn.Sequential(
        #     nn.Linear(1, 200000), nn.ReLU(), nn.Dropout(self.dropout),
        #     nn.Linear(200000,d_model)
        # )

        self.emb_q = nn.Embedding(200000,self.embed_l).to(device)# question embedding

        
        # self.emb_c = nn.Sequential(
        #     nn.Linear(7,1000), nn.ReLU(), nn.Dropout(self.dropout), # 7表示一个问题最大的concept数量
        #     nn.Linear(1000,d_model)
        # )

        self.emb_c = nn.Parameter(torch.randn(1000, self.embed_l).to(device), requires_grad=True)# kc embedding


        self.model = Architecture(n_question=n_question, n_blocks=n_blocks, n_heads=num_attn_heads, dropout=dropout,
                                    d_model=d_model, d_feature=d_model / num_attn_heads, d_ff=d_ff,  kq_same=self.kq_same, model_type=self.model_type, seq_len=seq_len)

        self.out = nn.Sequential(
            nn.Linear(d_model + self.embed_l,
                      final_fc_dim), nn.ReLU(), nn.Dropout(self.dropout),
            nn.Linear(final_fc_dim, final_fc_dim2), nn.ReLU(
            ), nn.Dropout(self.dropout),
            nn.Linear(final_fc_dim2, 1)
        )
        if emb_type.find("predc") != -1:
            self.qclasifier = nn.Sequential(
                nn.Linear(d_model + self.embed_l,
                        final_fc_dim), nn.ReLU(), nn.Dropout(self.dropout),
                nn.Linear(final_fc_dim, final_fc_dim2), nn.ReLU(
                ), nn.Dropout(self.dropout),
                nn.Linear(final_fc_dim2, self.n_pid)
            )
            
        if emb_type.find("pt") != -1: 
            self.t_out = nn.Sequential(
                nn.Linear(d_model + self.embed_l,
                        final_fc_dim), nn.ReLU(), nn.Dropout(self.dropout),
                nn.Linear(final_fc_dim, final_fc_dim2), nn.ReLU(
                ), nn.Dropout(self.dropout),
                nn.Linear(final_fc_dim2, 1)
            )

        self.reset()

    def reset(self):
        for p in self.parameters():
            if p.size(0) == self.n_pid+1 and self.n_pid > 0:
                torch.nn.init.constant_(p, 0.)

    def get_avg_skill_emb(self,c):
        # add zero for padding
        concept_emb_cat = torch.cat(
            [torch.zeros(1, self.embed_l).to(device), 
            self.emb_c], dim=0)
        # shift c

        related_concepts = (c+1).long()
        #[batch_size, seq_len, emb_dim]
        concept_emb_sum = concept_emb_cat[related_concepts, :].sum(
            axis=-2)

        #[batch_size, seq_len,1]
        concept_num = torch.where(related_concepts != 0, 1, 0).sum(
            axis=-1).unsqueeze(-1)
        concept_num = torch.where(concept_num == 0, 1, concept_num)
        concept_avg = (concept_emb_sum / concept_num)
        return concept_avg

    def forward(self, dcur, qtest=False, train=False, dgaps=None):
        q, c, r = dcur["qseqs"].long().to(device), dcur["cseqs"].long().to(device), dcur["rseqs"].long().to(device)
        qshft, cshft, rshft = dcur["shft_qseqs"].long().to(device), dcur["shft_cseqs"].long().to(device), dcur["shft_rseqs"].long().to(device)
        # print(f"q:{q.shape}")
        batch_size = q.size(0)

        dataset_id = dcur["dataset_id"].long().to(device)
        pid_data = torch.cat((q[:,0:1], qshft), dim=1) # shape[batch,200]
        q_data = torch.cat((c[:,0:1], cshft), dim=1) # shape[batch,200,7]
        target = torch.cat((r[:,0:1], rshft), dim=1)

        if self.emb_type.find("aug") != -1 and train:
            # new qids
            aug_pids= pid_data.unfold(1, self.q_window_size, 1)
            aug_pids = aug_pids.sum(dim=2)
            aug_pid_data = torch.where(aug_pids >= 200000, -1, aug_pids)

            # new cids
            if q_data.size(2) <  self.c_window_size:
                aug_cids = q_data.unfold(2, q_data.size(2), 1).sum(dim=-1)
            else:
                aug_cids = q_data.unfold(2, self.c_window_size, 1).sum(dim=-1)
            aug_cid_data = torch.where(aug_cids >= 1000, -1, aug_cids)
            # print(f"aug_cid_data:{aug_cid_data.shape}")

            # new rids
            window_counts = target.unfold(1, self.q_window_size, 1).sum(dim=2)
            aug_target_data = (window_counts > self.q_window_size / 2).float().long()

            # padding
            pad_dims = [0, pid_data.size(1) - aug_pid_data.size(1), 0, pid_data.size(0) - aug_pid_data.size(0)]
            tmp_aug_pid_data_padded = F.pad(aug_pid_data, pad_dims, value=-1)
            aug_pid_data_padded = torch.where(tmp_aug_pid_data_padded == -1, 0, tmp_aug_pid_data_padded)
            # print(f"aug_pid_data_padded:{aug_pid_data_padded.shape}")
            cid_padding_size = q_data.size(2) - aug_cid_data.size(2)
            aug_cid_data_padded = F.pad(aug_cid_data, (0, cid_padding_size))
            # print(f"aug_cid_data_padded:{aug_cid_data_padded.shape}")
            aug_target_data = F.pad(aug_target_data, pad_dims, value=0)

            # stack data
            all_pid_data = torch.cat([pid_data, aug_pid_data_padded],dim=0)
            # print(f"all_pid_data:{torch.max(all_pid_data)}")
            all_cid_data = torch.cat([q_data, aug_cid_data_padded],dim=0)
            # print(f"all_cid_data:{torch.max(all_cid_data)}")
            all_target_data = torch.cat([target, aug_target_data],dim=0)
            # print(f"all_target_data:{torch.max(all_target_data)}")

            # embedding
            emb_q = self.emb_q(all_pid_data) #[batch,max_len-1,emb_size]
            emb_c = self.get_avg_skill_emb(all_cid_data) #[batch,max_len-1,emb_size]
            dataset_embed_data = self.dataset_emb(dataset_id).unsqueeze(1).repeat(2,1,1)
            qa_embed_data = self.qa_embed(all_target_data)
        else:
            emb_q = self.emb_q(pid_data)#[batch,max_len-1,emb_size]
            emb_c = self.get_avg_skill_emb(q_data)#[batch,max_len-1,emb_size]
            dataset_embed_data = self.dataset_emb(dataset_id).unsqueeze(1)
            # print(f"dataset_embed_data:{dataset_embed_data.shape}")
            try:
                qa_embed_data = self.qa_embed(target)
            except:
                print(f"target:{target}")
        
        if self.emb_type.find("pt") != -1:
            sg, sgshft = dgaps["sgaps"].long(), dgaps["shft_sgaps"].long()
            s_gaps = torch.cat((sg[:, 0:1], sgshft), dim=1)
            emb_t = self.time_emb(s_gaps)
            q_embed_data += emb_t
        
        # print(f"emb_q:{emb_q.shape}")
        # print(f"emb_c:{emb_c.shape}")
        # print(f"dataset_embed_data:{dataset_embed_data.shape}")
        q_embed_data = emb_q + emb_c + dataset_embed_data
        qa_embed_data = q_embed_data + qa_embed_data

        # BS.seqlen,d_model
        y2, y3 = 0, 0
        d_output = self.model((q_embed_data, qa_embed_data))
        concat_q = torch.cat([d_output, q_embed_data], dim=-1)
        output = self.out(concat_q).squeeze(-1)
        m = nn.Sigmoid()
        preds = m(output)

        cl_losses = 0
        if self.emb_type.find("predc") != -1 and train:
            sm = dcur["smasks"].long()
            start = 0
            cpreds = self.qclasifier(concat_q[:,start:,:])
            # print(f"cpreds:{cpreds.shape}")
            flag = sm[:,start:]==1
            # print(f"flag:{flag.shape}")
            # print(f"cpreds:{cpreds[:,:-1,:][flag].shape}")
            # print(f"qtag:{q[:,start:][flag].shape}")
            cl_loss = self.ce_loss(cpreds[:,:-1,:][flag], q[:,start:][flag])
            cl_losses += self.cf_weight * cl_loss
        
        if self.emb_type.find("pt") != -1 and train:
            t_label= dgaps["shft_pretlabel"].double()
            t_combined = torch.cat((d_output, emb_t), -1)
            t_output = self.t_out(t_combined).squeeze(-1)
            t_pred = m(t_output)[:,1:]
            # print(f"t_pred:{t_pred}")
            sm = dcur["smasks"]
            ty = torch.masked_select(t_pred, sm)
            # print(f"min_y:{torch.min(ty)}")
            tt = torch.masked_select(t_label, sm)
            # print(f"min_t:{torch.min(tt)}")
            t_loss = binary_cross_entropy(ty.double(), tt.double())
            # t_loss = mse_loss(ty.double(), tt.double())
            # print(f"t_loss:{t_loss}")
            cl_losses += self.t_weight * t_loss
        if self.emb_type.find("aug") != -1 and train:
            # cal loss of augmented data
            aug_preds = preds[batch_size:]
            # 拆分预测结果
            preds = preds[:batch_size]

            mask_select = tmp_aug_pid_data_padded != -1
            select_preds = aug_preds[mask_select]
            select_targets = aug_target_data[mask_select]
            # print(f"select_preds:{torch.max(select_preds)}")
            # print(f"select_targets:{torch.max(select_targets)}")
            cl_losses = self.ce_loss(select_preds, select_targets.float())
            # print(f"cl_losses:{cl_losses}")
  
        if train:
            if self.emb_type == "qid":
                return preds, y2, y3
            else:
                return preds, y2, y3, cl_losses
        else:
            if qtest:
                return preds, concat_q
            else:
                return preds

class Architecture(nn.Module):
    def __init__(self, n_question,  n_blocks, d_model, d_feature,
                 d_ff, n_heads, dropout, kq_same, model_type, seq_len):
        super().__init__()
        """
            n_block : number of stacked blocks in the attention
            d_model : dimension of attention input/output
            d_feature : dimension of input in each of the multi-head attention part.
            n_head : number of heads. n_heads*d_feature = d_model
        """
        self.d_model = d_model
        self.model_type = model_type

        if model_type in {'gpt4kt','unikt'}:
            self.blocks_2 = nn.ModuleList([
                TransformerLayer(d_model=d_model, d_feature=d_model // n_heads,
                                 d_ff=d_ff, dropout=dropout, n_heads=n_heads, kq_same=kq_same)
                for _ in range(n_blocks)
            ])
        self.position_emb = CosinePositionalEmbedding(d_model=self.d_model, max_len=seq_len)

    def forward(self, inputs):
        # target shape  bs, seqlen
        q_embed_data, qa_embed_data = inputs
        seqlen, batch_size = q_embed_data.size(1), q_embed_data.size(0)

        q_posemb = self.position_emb(q_embed_data)
        q_embed_data = q_embed_data + q_posemb
        qa_posemb = self.position_emb(qa_embed_data)
        qa_embed_data = qa_embed_data + qa_posemb

        qa_pos_embed = qa_embed_data
        q_pos_embed = q_embed_data

        y = qa_pos_embed
        seqlen, batch_size = y.size(1), y.size(0)
        x = q_pos_embed

        # encoder
        
        for block in self.blocks_2:
            # x.requires_grad_(True)
            # y.requires_grad_(True)
            # def run_block(mask, query, key, values, apply_pos):
            #     return block(mask, query, key, values, apply_pos)
            # x = checkpoint(run_block, mask, x, x, y, apply_pos)
            
            x = checkpoint(block, x, x, y)
            # x = block(mask=0, query=x, key=x, values=y, apply_pos=True) # True: +FFN+残差+laynorm 非第一层与0~t-1的的q的attention, 对应图中Knowledge Retriever
            # mask=0，不能看到当前的response, 在Knowledge Retrever的value全为0，因此，实现了第一题只有question信息，无qa信息的目的
            # print(x[0,0,:])
            # x = input_data[1]
        return x

class TransformerLayer(nn.Module):
    def __init__(self, d_model, d_feature,
                 d_ff, n_heads, dropout,  kq_same):
        super().__init__()
        """
            This is a Basic Block of Transformer paper. It containts one Multi-head attention object. Followed by layer norm and postion wise feedforward net and dropout layer.
        """
        kq_same = kq_same == 1
        # Multi-Head Attention Block
        self.masked_attn_head = MultiHeadAttention(
            d_model, d_feature, n_heads, dropout, kq_same=kq_same)

        # Two layer norm layer and two droput layer
        self.layer_norm1 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)

        self.linear1 = nn.Linear(d_model, d_ff)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ff, d_model)

        self.layer_norm2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, query, key, values):
        """
        Input:
            block : object of type BasicBlock(nn.Module). It contains masked_attn_head objects which is of type MultiHeadAttention(nn.Module).
            mask : 0 means, it can peek only past values. 1 means, block can peek only current and pas values
            query : Query. In transformer paper it is the input for both encoder and decoder
            key : Keys. In transformer paper it is the input for both encoder and decoder
            Values. In transformer paper it is the input for encoder and  encoded output for decoder (in masked attention part)

        Output:
            query: Input gets changed over the layer and returned.

        """
        mask = 0
        apply_pos=True
        seqlen, batch_size = query.size(1), query.size(0)
        nopeek_mask = np.triu(
            np.ones((1, 1, seqlen, seqlen)), k=mask).astype('uint8')
        src_mask = (torch.from_numpy(nopeek_mask) == 0).to(device)
        if mask == 0:  # If 0, zero-padding is needed.
            # Calls block.masked_attn_head.forward() method
            query2 = self.masked_attn_head(
                query, key, values, mask=src_mask, zero_pad=True) # 只能看到之前的信息，当前的信息也看不到，此时会把第一行score全置0，表示第一道题看不到历史的interaction信息，第一题attn之后，对应value全0
        else:
            # Calls block.masked_attn_head.forward() method
            query2 = self.masked_attn_head(
                query, key, values, mask=src_mask, zero_pad=False)

        query = query + self.dropout1((query2)) # 残差1
        query = self.layer_norm1(query) # layer norm
        if apply_pos:
            query2 = self.linear2(self.dropout( # FFN
                self.activation(self.linear1(query))))
            query = query + self.dropout2((query2)) # 残差
            query = self.layer_norm2(query) # lay norm
        return query


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, d_feature, n_heads, dropout, kq_same, bias=True):
        super().__init__()
        """
        It has projection layer for getting keys, queries and values. Followed by attention and a connected layer.
        """
        self.d_model = d_model
        self.d_k = d_feature
        self.h = n_heads
        self.kq_same = kq_same

        self.v_linear = nn.Linear(d_model, d_model, bias=bias)
        self.k_linear = nn.Linear(d_model, d_model, bias=bias)
        if kq_same is False:
            self.q_linear = nn.Linear(d_model, d_model, bias=bias)
        self.dropout = nn.Dropout(dropout)
        self.proj_bias = bias
        self.out_proj = nn.Linear(d_model, d_model, bias=bias)

        self._reset_parameters()

    def _reset_parameters(self):
        xavier_uniform_(self.k_linear.weight)
        xavier_uniform_(self.v_linear.weight)
        if self.kq_same is False:
            xavier_uniform_(self.q_linear.weight)

        if self.proj_bias:
            constant_(self.k_linear.bias, 0.)
            constant_(self.v_linear.bias, 0.)
            if self.kq_same is False:
                constant_(self.q_linear.bias, 0.)
            constant_(self.out_proj.bias, 0.)

    def forward(self, q, k, v, mask, zero_pad):

        bs = q.size(0)

        # perform linear operation and split into h heads

        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        if self.kq_same is False:
            q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        else:
            q = self.k_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)

        # transpose to get dimensions bs * h * sl * d_model

        k = k.transpose(1, 2)
        q = q.transpose(1, 2)
        v = v.transpose(1, 2)
        # calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k,
                           mask, self.dropout, zero_pad)

        # concatenate heads and put through final linear layer
        concat = scores.transpose(1, 2).contiguous()\
            .view(bs, -1, self.d_model)

        output = self.out_proj(concat)

        return output


def attention(q, k, v, d_k, mask, dropout, zero_pad):
    """
    This is called by Multi-head atention object to find the values.
    """
    # d_k: 每一个头的dim
    scores = torch.matmul(q, k.transpose(-2, -1)) / \
        math.sqrt(d_k)  # BS, 8, seqlen, seqlen
    bs, head, seqlen = scores.size(0), scores.size(1), scores.size(2)

    scores.masked_fill_(mask == 0, -1e32)
    scores = F.softmax(scores, dim=-1)  # BS,8,seqlen,seqlen
    # print(f"before zero pad scores: {scores.shape}")
    # print(zero_pad)
    if zero_pad:
        pad_zero = torch.zeros(bs, head, 1, seqlen).to(device)
        scores = torch.cat([pad_zero, scores[:, :, 1:, :]], dim=2) # 第一行score置0
    # print(f"after zero pad scores: {scores}")
    scores = dropout(scores)
    output = torch.matmul(scores, v)
    # import sys
    # sys.exit()
    return output


class LearnablePositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super().__init__()
        # Compute the positional encodings once in log space.
        pe = 0.1 * torch.randn(max_len, d_model)
        pe = pe.unsqueeze(0)
        self.weight = nn.Parameter(pe, requires_grad=True)

    def forward(self, x):
        return self.weight[:, :x.size(Dim.seq), :]  # ( 1,seq,  Feature)


class CosinePositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super().__init__()
        # Compute the positional encodings once in log space.
        pe = 0.1 * torch.randn(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).long()
        div_term = torch.exp(torch.arange(0, d_model, 2).long() *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.weight = nn.Parameter(pe, requires_grad=False)

    def forward(self, x):
        return self.weight[:, :x.size(Dim.seq), :]  # ( 1,seq,  Feature)


In [10]:
unikt = UNIKT()
print("emb_c in model.parameters:", any(p is unikt.emb_c for p in unikt.parameters()))
print("emb_c in model.state_dict:", 'emb_c' in unikt.state_dict())

model_name: unikt, emb_type: qid
emb_c in model.parameters: True
emb_c in model.state_dict: True
